<a href="https://colab.research.google.com/github/Jules-4LL4RT/Internship/blob/main/IDS_Test_CICIDS2017.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
%tensorflow_version 2.x
import tensorflow
from tensorflow.keras.utils import get_file
import os
import io
import re
import csv
from time import time
from sklearn import metrics
from scipy.stats import zscore
from google.colab import files
#from google.colab import drive
#drive.mount('/content/drive')
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping
import ipaddress

t0 = time()
try:
     path = get_file('newFridayAfternoonDDoSaa.csv', origin=\
     'https://raw.githubusercontent.com/Jules-4LL4RT/Internship/main/newFridayAfternoonDDoSaa.csv',archive_format=None)
except:
    print('Something went wrong, please try again')
    raise

#path = 'https://raw.githubusercontent.com/Jules-4LL4RT/Internship/main/newFridayMorning2aa.csv'

print(path)
df = pd.read_csv(path, sep=',',error_bad_lines=True)
#df.to_csv("newFridayMorning3.csv")
#df = pd.read_csv(path)

#Display amount of rows and columns
print(df.shape)
#Check we read all rows (lines) of the given dataset (Same as previous basically)
print("Read {} rows.".format(len(df)))

#Sample X % of the dataset
#Replace=false because same row can't be sampled 2 times
df = df.sample(frac=0.50, replace=False) 

#Drop columns with missing values
df.dropna(inplace=True,axis=1)

excluded = ['  Fwd Header Length']
df = df.drop(columns=excluded, errors='ignore')

#Removing doubled columns
#df.columns = df.columns.str.strip()
#df = df.drop(columns=['Fwd Header Length.1'])
#print(df.shape)

#Display n columns and rows maximum
#pd.set_option('display.max_columns', 6)
#pd.set_option('display.max_rows', 6)
#df[0:2]

 
#Printing columns (titles, quantity...)

def analyze(df):
    print()
    cols = df.columns.values
    total = float(len(df))

    print("{} rows".format(int(total)))
    for col in cols:
        #Count amount of unique columbs
        i=0
        nbCol = df[col].unique()
        unique_count = len(nbCol)
        #Display columns titles
        print("**{}:{} ({}%)".format(col,unique_count,\
        int(((unique_count)/total)*100)))

analyze(df)

#SD stands for standard deviation
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd

#Dummy variables (ex: temp can be hot, medium or cold)
#Axis=1 means columns
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = f"{name}-{x}"
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)

#Encoding depends of data types
for name in df.columns:
  if name == ' Label':
    pass
  elif name in [' Source IP',' Destination IP', 'Flow ID',' Timestamp']:
    encode_text_dummy(df,name)
  #elif name in [' Source IP',' Destination IP']:
    #name = ipaddress.ip_address(name)
    #encode_numeric_zscore(df,name)
  #elif name in ['Flow ID',' Timestamp']:
    #encode_text_dummy(df,name)
  else:
    encode_numeric_zscore(df,name)

#'Flow ID',' Source IP',' Destination IP',' Timestamp'  

#print(df.columns.tolist())

#Remove columns with missing values
df.dropna(inplace=True,axis=1)

#Classification
x_columns = df.columns.drop(' Label')
x = df[x_columns].values
dummies = pd.get_dummies(df[' Label'])
outcomes = dummies.columns
num_classes = len(outcomes)
y = dummies.values

df.groupby(' Label')[' Label'].count()

#print(num_classes)
#print(df.dtypes)

t1 = time()

# Test/train split - X% test
# Split into train/test
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.20, random_state=13)

#Converting an input as an array (normally not needed)
#x_train=np.asarray(x_train).astype(np.int_)
#y_train=np.asarray(y_train).astype(np.int_)

#Defining the model
model = Sequential()
#(Hidden) Layers
#The first number is the node amount, input the number of variables
#ReLU is for rectified linear unit activation function
model.add(Dense(10, input_dim=x.shape[1], activation='relu'))
model.add(Dense(50, input_dim=x.shape[1], activation='relu'))
model.add(Dense(10, input_dim=x.shape[1], activation='relu'))
model.add(Dense(1, kernel_initializer='normal'))
model.add(Dense(y.shape[1],activation='softmax'))
#loss function are big if prediction deviation is high
#loss='categorical_crossentropy' for multiple output (good for classification tasks)
model.compile(loss='categorical_crossentropy', optimizer='adam')
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, 
                        patience=5, verbose=1, mode='auto',
                           restore_best_weights=True)
model.fit(x_train,y_train,validation_data=(x_test,y_test),
          callbacks=[monitor],verbose=2,epochs=50)
          #Verbose is graphical mode for epoch, 2 means one line per epoch
          #Callbacks are to avoid overfitting, monitoring val_loss
          #Patience is the amount of epochs after which we stop if no improvement

#Accuracy
prediction = model.predict(x_test)
prediction = np.argmax(prediction,axis=1)
y_eval = np.argmax(y_test,axis=1)

score = metrics.accuracy_score(y_eval, prediction)
print("Score: {}".format(score))

print(time() - t0)
print(time() - t1)



18874368/18860544 [==============================] - 0s 0us/step
/root/.keras/datasets/newFridayAfternoonDDoSaa.csv


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


(44999, 85)
Read 44999 rows.

22500 rows
**Flow ID:17271 (76%)
** Source IP:622 (2%)
** Source Port:14056 (62%)
** Destination IP:955 (4%)
** Destination Port:3023 (13%)
** Protocol:3 (0%)
** Timestamp:30 (0%)
** Flow Duration:18373 (81%)
** Total Fwd Packets:152 (0%)
** Total Backward Packets:162 (0%)
**Total Length of Fwd Packets:1546 (6%)
** Total Length of Bwd Packets:2015 (8%)
** Fwd Packet Length Max:922 (4%)
** Fwd Packet Length Min:98 (0%)
** Fwd Packet Length Mean:1944 (8%)
** Fwd Packet Length Std:2173 (9%)
**Bwd Packet Length Max:830 (3%)
** Bwd Packet Length Min:259 (1%)
** Bwd Packet Length Mean:2218 (9%)
** Bwd Packet Length Std:2232 (9%)
** Flow Packets/s:18655 (82%)
** Flow IAT Mean:18630 (82%)
** Flow IAT Std:15677 (69%)
** Flow IAT Max:14778 (65%)
** Flow IAT Min:2587 (11%)
**Fwd IAT Total:9187 (40%)
** Fwd IAT Mean:11008 (48%)
** Fwd IAT Std:9981 (44%)
** Fwd IAT Max:9085 (40%)
** Fwd IAT Min:2094 (9%)
**Bwd IAT Total:9462 (42%)
** Bwd IAT Mean:11147 (49%)
** Bwd IAT

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:96: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


KeyboardInterrupt: ignored